In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/PorterFarms/')
print("============================================")
print("/  CleanChannel is running.               /")
print("============================================")
import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_SLACK_TOKEN = config['slack']['token']
_CHIRPSTACK_USER = config['chirpstack']['user']
_CHIRPSTACK_PASS = config['chirpstack']['password']

_DB_HOST = config['kanjidb']['dbhost']
_DB_PORT = config['kanjidb']['dbport']
_DB_NAME = config['kanjidb']['dbname']
_DB_USER = config['kanjidb']['dbuser']
_DB_PASS  = config['kanjidb']['dbpass']

_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, _SLACK_TOKEN)
logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

updatetemplate = "{\
    \"ok\": true, \
    \"channel\": \"C024BE91L\", \
    \"ts\": \"1401383885.000061\", \
    \"text\": \"Updated text you carefully authored\", \
    \"message\": { \
        \"text\": \"Updated text you carefully authored\", \
        \"user\": \"DGM50H51N\" \
    } }"

/  CleanChannel is running.               /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [5]:
def updateMessageToSlack(slackchannel, ts, text):
    sc = SlackClient(_SLACK_TOKEN)
    response = sc.api_call("chat.update", channel=slackchannel, ts=ts, text=text, blocks=[])
    #print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(blockmessage)
      print(response)
      return 'failed'
    else:
      print("Ok posting message to Slack channel")
      return 'ok'

In [6]:
STATUS_CLOSED   = 10003
STATUS_ARCHIVED = 10004

now = datetime.now() + timedelta(hours = 4)

closedticketquery = "SELECT * FROM kanji_ticket WHERE status_id={}".format(STATUS_CLOSED)
#logger(_LOG_DEBUG, closedticketquery)
df = pd.read_sql(closedticketquery, conn)

logger(_LOG_DEBUG, "number of closed tickets {}".format(len(df.index)))
for ind in df.index:
    logger(_LOG_DEBUG, ind)
    blockmessage = json.loads(updatetemplate)
    idticket = df['idticket'][ind]
    ticketts = df['ticketts'][ind]
    tickettypeid = df['type_id'][ind]
    ticketnodeid = df['node_id'][ind]
    ticketTypeQuery = "SELECT description FROM kanji_tickettype WHERE idtickettype={}".format(tickettypeid)
    df2 = pd.read_sql(ticketTypeQuery, conn)
    ticketTypeDescription = df2['description'][0]
    nodeQuery = "SELECT node.name, location.description, location.slackchannelid FROM kanji_node node JOIN kanji_location location ON node.location_id=location.idlocation WHERE idnode={}".format(ticketnodeid)
    logger(_LOG_DEBUG, nodeQuery)
    df3 = pd.read_sql(nodeQuery, conn)
    nodeName = df3["name"][0]
    nodeLocation = df3["description"][0]
    logger(_LOG_DEBUG, "ticketChannel {}".format(df['ticketchannel'][ind]))
    if df['ticketchannel'][ind]!='infrastructure':
      ticketchannel = df3["slackchannelid"][0]
    else:
      ticketchannel = "C013YFJF276"
    logger(_LOG_DEBUG, "ticketchannel {}".format(ticketchannel))
    blockmessage['channel'] = ticketchannel
    blockmessage['ts'] = ticketts
    msgClosedText = "*`#{} RESOLVED* -- {} on {} at {}".format(idticket, ticketTypeDescription, nodeName, nodeLocation)
    if updateMessageToSlack(ticketchannel, ticketts, msgClosedText) == 'ok':
      logger(_LOG_INFO, "Archiving ticket {}".format(idticket))
      kt.updateTicketStatus(conn, idticket, STATUS_ARCHIVED)
 

Ok posting message to Slack channel
Archiving ticket 302
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=302
Ok posting message to Slack channel
Archiving ticket 288
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=288
Ok posting message to Slack channel
Archiving ticket 293
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=293
Ok posting message to Slack channel
Archiving ticket 294
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=294
Ok posting message to Slack channel
Archiving ticket 290
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=290
Ok posting message to Slack channel
Archiving ticket 289
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=289
Ok posting message to Slack channel
Archiving ticket 295
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=295
Ok posting message to Slack channel
Archiving ticket 292
UPDATE kanji_ticket SET status_id=10004 WHERE idticket=292
Ok posting message to Slack channel
Archiving ticket 291
UPDATE kanji_ti